In [2]:
import boto3
import io
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

# export to S-3
import awswrangler as wr

# Imputation
from sklearn.impute import SimpleImputer


plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#!pip install awswrangler

In [40]:
pd.set_option('display.max_rows', 200)

### Load a consolidated file from S-3

In [5]:
session = boto3.Session()
s3 = session.client('s3')

In [6]:
bucket_name = 'ppsg-collections'
file_name = '2021-06-01/consolidated/random_data/consolidated_sample.parquet'

In [7]:
%%time
obj = s3.get_object(Bucket = bucket_name, Key = file_name)
df = pd.read_parquet(io.BytesIO(obj['Body'].read())) 

CPU times: user 2min 4s, sys: 3min 41s, total: 5min 45s
Wall time: 1min 52s


In [6]:
df = df.fillna(value=np.nan)

### Set Processing periods

In [7]:
start_date = '2020-02-01'
end_date = '2021-04-01'

#### Choose data periods

In [8]:
df= df[(df['nextpaymentduedt']>=start_date) & (df['nextpaymentduedt']<=end_date)]

In [9]:
# remove 2020 April data
df =  df[df['nextpaymentduedt'] != '2020-04-01']

### Import Functions

In [10]:
from important_functions import processing_data

### Processing Data (Rollback  --> Expand -- > Add one more row to handle Sunday)

In [36]:
%%time
df = processing_data (df,method = 'add row')

CPU times: user 51min 6s, sys: 9min 28s, total: 1h 34s
Wall time: 1h 14s


In [11]:
%%time
import awswrangler as wr
wr.s3.to_parquet(
    df=df,
    path='s3://ppsg-collections/2021-06-01/consolidated/random_data/consolidated_sample_expanded.parquet'
)

CPU times: user 2min 44s, sys: 23.5 s, total: 3min 7s
Wall time: 2min 43s


{'paths': ['s3://ppsg-collections/2021-06-01/consolidated/random_data/consolidated_sample_expanded.parquet'],
 'partitions_values': {}}

In [42]:
#df.shape

(28488619, 153)

In [44]:
#df.head()

,loanid,loanstatusid,nextpaymentduedt,rundt,ndd,rundatenumber,priormeperiod,MEPeriod,loanstatus,priorstatus,...,apprbucket,amorttype,termbucket,CRAFlag,rollback,PastDue,days_in_month,PmtDt,PaidDt,Down2
0,1000017538,A,2020-02-01,2020-02-02,20200201.0,20200202.0,202001.0,202001.0,P,1,...,Other,Fixed,30 yr,N,0.0,1,29,2020-03-03,31,1
1,1000017538,A,2020-02-01,2020-02-03,20200201.0,20200203.0,202001.0,202001.0,P,1,...,Other,Fixed,30 yr,N,0.0,2,29,2020-03-03,31,1
2,1000017538,A,2020-02-01,2020-02-04,20200201.0,20200204.0,202001.0,202001.0,P,1,...,Other,Fixed,30 yr,N,0.0,3,29,2020-03-03,31,1
3,1000017538,A,2020-02-01,2020-02-05,20200201.0,20200205.0,202001.0,202001.0,P,1,...,Other,Fixed,30 yr,N,0.0,4,29,2020-03-03,31,1
4,1000017538,A,2020-02-01,2020-02-06,20200201.0,20200206.0,202001.0,202001.0,P,1,...,Other,Fixed,30 yr,N,0.0,5,29,2020-03-03,31,1
